In [1]:
import pandas as pd
import numpy as np  
import os
from pathlib import Path
from collections import defaultdict
notebook_dir = Path(os.getcwd())    

# Experiment progress table

## Single transition: NSMDP consists of only two stationary MDPs

### With Notification

| Gym Environment | DDQN | Vanilla MCTS | PAMCTS | AlphaZero | ADA-MCTS | RATs | Pro-OLS | Pro-WLS |
|-----------------|------|--------------|--------|-----------|----------|------|---------|---------|
| CartPole - varying mass        | N/A     |   DONE   |        |           |          |      |         |         |
| CartPole - varying gravity      |    N/A  |   DONE   |        |           |          |      |         |         |
| MountainCar     |    N/A  |              |        |           |          |      |         |         |
| Pendulum        |     N/A |             |        |           |          |      |         |         |
| Acrobot         |   N/A   |             |        |           |          |      |         |         |
| FrozenLake      |   N/A  |       DONE       |        |           |          |      |         |         |
| CliffWalker     |      |              |        |           |          |      |         |         |
| Bridge          |      |   ****           |        |           |          |      |         |         |

### Without Notification


| Gym Environment | DDQN | Vanilla MCTS | PAMCTS | AlphaZero | ADA-MCTS | RATs | Pro-OLS | Pro-WLS |
|-----------------|------|--------------|--------|-----------|----------|------|---------|---------|
| CartPole - varying mass        |      |   DONE   |        |           |          |      |         |         |
| CartPole - varying gravity      |      |   DONE   |        |           |          |      |         |         |
| MountainCar     |      |              |        |           |          |      |         |         |
| Pendulum        |      |              |        |           |          |      |         |         |
| Acrobot         |      |              |        |           |          |      |         |         |
| FrozenLake      |   DONE   |       DONE       |        |           |          |      |         |         |
| CliffWalker     |      |              |        |           |          |      |         |         |
| Bridge          |      |              |        |           |          |      |         |         |


## Continious changes: At each MDP step the NSMDP is updtaded to a new stationary MDP (RATs assumption)

### With Notification

| Gym Environment | DDQN | Vanilla MCTS | PAMCTS | AlphaZero | ADA-MCTS | RATs | Pro-OLS | Pro-WLS |
|-----------------|------|--------------|--------|-----------|----------|------|---------|---------|
| CartPole        |      |              |        |           |          |      |         |         |
| MountainCar     |      |              |        |           |          |      |         |         |
| Pendulum        |      |              |        |           |          |      |         |         |
| Acrobot         |      |              |        |           |          |      |         |         |
| FrozenLake      | DONE     |  DONE            |        |           |          |      |         |         |
| CliffWalker     |      |              |        |           |          |      |         |         |
| Bridge          |      |              |        |           |          |      |         |         |


### Without Notification


| Gym Environment | DDQN | Vanilla MCTS | PAMCTS | AlphaZero | ADA-MCTS | RATs | Pro-OLS | Pro-WLS |
|-----------------|------|--------------|--------|-----------|----------|------|---------|---------|
| CartPole        |      |              |        |           |          |      |         |         |
| MountainCar     |      |              |        |           |          |      |         |         |
| Pendulum        |      |              |        |           |          |      |         |         |
| Acrobot         |      |              |        |           |          |      |         |         |
| FrozenLake      | DONE     |  DONE            |        |           |          |      |         |         |
| CliffWalker     |      |              |        |           |          |      |         |         |
| Bridge          |      |              |        |           |          |      |         |         |




# Vanilla MCTS

## Frozen Lake

### MCTS single transition probability update **without** notification 

Experiment code: experiments/NS_FrozenLake/single_discrete_transition.py

MCTS on NS_FrozenLake. Initial transition prob is [0.7,0.15,0.15], where the first element of the list is the intended direction and the other two are the perpendicular directions. At MDP step t = 0 the simulator updates the transition prob to [p, (1-p)/2,(1-p)/2]. The MCTS planning agent only has access to MDP_0 (transition probs =[0.7,0.15,0.15]) while the "realworld" envirment continues according to [p, (1-p)/2,(1-p)/2].


### Vanilla MCTS single transiton with Negative rewards **without** notification

This trial should be in working order. I use my own reimplementation of MCTS with chance nodes and modify the rewards so that holes have an imediate reward of -1. Ran on only 3000 iterations inst

In [2]:
exp_path = notebook_dir / 'NS_FrozenLake'/ 'results' / "VanillaMCTS_withoutChangNotif_withNegative_Rewards_2024-05-14_02:44:06.csv"
frozen_lake_single_transition_withoutNotification_withNegativeRewars = pd.read_csv(exp_path)
results = frozen_lake_single_transition_withoutNotification_withNegativeRewars.groupby("p")['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = frozen_lake_single_transition_withoutNotification_withNegativeRewars.groupby("p")['total_time'].agg(['mean']) / 60
results["num MCTS iterations"] = frozen_lake_single_transition_withoutNotification_withNegativeRewars.groupby("p")['num_iter'].agg(['mean'])

In [3]:
results

,mean,std,sum,count,std error,Mean Episode Time in Minutes,num MCTS iterations
p,,,,,,,
0.4,-0.76,0.653197,-76.0,100,0.065320,1.998520,3000.0
0.5,-0.64,0.772246,-64.0,100,0.077225,1.700917,3000.0
0.6,-0.24,0.975663,-24.0,100,0.097566,2.324237,3000.0
0.8,0.34,0.945163,34.0,100,0.094516,2.498202,3000.0
0.9,0.72,0.697470,72.0,100,0.069747,2.549106,3000.0
1.0,1.00,0.000000,100.0,100,0.000000,1.974305,3000.0


### Vanilla MCTS single transiton with Negative rewards **with** notification

In [4]:
exp_path = notebook_dir / 'NS_FrozenLake'/ 'results' / "VanillaMCTS_withChangNotif_withNegative_Rewards_2024-05-14_12:14:23.csv"
frozen_lake_single_transition_withNotification_withNegativeRewars = pd.read_csv(exp_path)
results = frozen_lake_single_transition_withNotification_withNegativeRewars.groupby("p")['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = frozen_lake_single_transition_withNotification_withNegativeRewars.groupby("p")['total_time'].agg(['mean']) / 60
results["num MCTS iterations"] = frozen_lake_single_transition_withNotification_withNegativeRewars.groupby("p")['num_iter'].agg(['mean'])

In [5]:
results

,mean,std,sum,count,std error,Mean Episode Time in Minutes,num MCTS iterations
p,,,,,,,
0.4,-0.74,0.675995,-74.0,100,0.067600,1.982085,3000.0
0.5,-0.76,0.653197,-76.0,100,0.065320,2.025658,3000.0
0.6,-0.18,0.988622,-18.0,100,0.098862,2.247237,3000.0
0.8,0.44,0.902522,44.0,100,0.090252,2.459791,3000.0
0.9,0.68,0.736906,68.0,100,0.073691,2.472721,3000.0
1.0,1.00,0.000000,100.0,100,0.000000,1.976788,3000.0


### Vanilla MCTS continuous transitions - negative rewards - without notification

In [6]:
exp_path = notebook_dir / "NS_FrozenLake"/ "results" / "MCTS_ContinuousUpdate_withoutChangNotif_2024-05-21_16:11:11.csv"
frozen_lake_continuous_update_withoutNotification = pd.read_csv(exp_path)
frozen_lake_continuous_update_withoutNotification.columns



Index(['sample_id', 'reward', 'experiment_name', 'gamma', 'c', 'num_iter',
       'total_time', 'seed'],
      dtype='object')

In [7]:
results = frozen_lake_continuous_update_withoutNotification.groupby("num_iter")['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = frozen_lake_continuous_update_withoutNotification.groupby("num_iter")['total_time'].agg(['mean']) / 60
results

,mean,std,sum,count,std error,Mean Episode Time in Minutes
num_iter,,,,,,
25,-0.84,0.545320,-84.0,100,0.054532,0.021290
100,-0.86,0.512865,-86.0,100,0.051286,0.098224
1000,-0.88,0.477367,-88.0,100,0.047737,3.435035
3000,-0.90,0.438086,-90.0,100,0.043809,24.214279


### Vanilla MCTS continuous transitions - positive rewards - with notification

In [8]:
exp_path = notebook_dir / "NS_FrozenLake"/ "results" / "MCTS_ContinuousUpdate_with_ChangNotif_2024-05-17_22:43:11.csv"
MCTS_frozen_lake_continuous_update_with_Notification = pd.read_csv(exp_path)

results = MCTS_frozen_lake_continuous_update_with_Notification.groupby("num_iter")['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = MCTS_frozen_lake_continuous_update_with_Notification.groupby("num_iter")['total_time'].agg(['mean']) / 60
results

,mean,std,sum,count,std error,Mean Episode Time in Minutes
num_iter,,,,,,
25,0.10,0.301511,10.0,100,0.030151,0.028349
50,0.12,0.326599,12.0,100,0.032660,0.045069
100,0.17,0.377525,17.0,100,0.037753,0.071810
500,0.18,0.386123,18.0,100,0.038612,0.285879
1000,0.22,0.416333,22.0,100,0.041633,0.555850
3000,0.27,0.446196,27.0,100,0.044620,1.333811


### DDQN Single discrete change - without notification

In [9]:
exp_path = notebook_dir / "NS_FrozenLake"/ "results" / "DDQN_singe_discrete_change_withoutChangNotif_2024-05-17_22:54:55.csv"

DDQN_frozen_lake_single_update_with_Notification = pd.read_csv(exp_path)

results = DDQN_frozen_lake_single_update_with_Notification.groupby("p")["reward"].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = DDQN_frozen_lake_single_update_with_Notification.groupby("p")['total_time'].agg(['mean']) / 60
results

,mean,std,sum,count,std error,Mean Episode Time in Minutes
p,,,,,,
0.4,0.22,0.416333,22.0,100,0.041633,0.011522
0.5,0.47,0.501614,47.0,100,0.050161,0.005588
0.6,0.66,0.476095,66.0,100,0.047610,0.000817
0.8,0.91,0.287623,91.0,100,0.028762,0.001661
0.9,0.93,0.256432,93.0,100,0.025643,0.002758
1.0,0.00,0.000000,0.0,100,0.000000,0.028574


### DDQN Continuos discrete change - without notification

In [10]:
exp_path = notebook_dir / "NS_FrozenLake"/ "results" / "DDQN_ContinuousUpdate_withoutChangNotif_2024-05-17_22:55:12.csv"

DDQN_frozen_lake_continuous_update_with_Notification = pd.read_csv(exp_path)
results = DDQN_frozen_lake_continuous_update_with_Notification["reward"].agg(['mean', 'std','sum','count'])


results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = DDQN_frozen_lake_continuous_update_with_Notification["total_time"].agg(['mean']) / 60

results = results.to_frame().T
results

,mean,std,sum,count,std error,Mean Episode Time in Minutes
reward,0.27,0.446196,27.0,100.0,0.04462,"mean 0.015483 Name: total_time, dtype: float64"


### AlphaZero Single update with change notification -- these aer sus..



In [11]:
exp_path = notebook_dir / "NS_FrozenLake"/ "results" / "AlphaZero_singe_discrete_change_withoutChangNotif_2024-05-24_01:14:09.csv"

AlphaZero_frozen_lake_single_update_with_Notification = pd.read_csv(exp_path)
AlphaZero_frozen_lake_single_update_with_Notification
results = AlphaZero_frozen_lake_single_update_with_Notification.groupby(["p","num_iter"])["reward"].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])

results["Mean Episode Time in Minutes"] = AlphaZero_frozen_lake_single_update_with_Notification.groupby(["p","num_iter"])['total_time'].agg(['mean']) / 60
results

mean       std   sum  count  std error  \
p   num_iter                                           
0.4 25        0.00  0.000000   0.0    100   0.000000   
    100       0.00  0.000000   0.0    100   0.000000   
    1000     -0.12  0.383498 -12.0    100   0.038350   
    3000     -0.92  0.393893 -92.0    100   0.039389   
0.5 25        0.00  0.000000   0.0    100   0.000000   
    100       0.00  0.000000   0.0    100   0.000000   
    1000     -0.12  0.326599 -12.0    100   0.032660   
    3000     -0.64  0.772246 -64.0    100   0.077225   
0.6 25        0.00  0.000000   0.0    100   0.000000   
    100       0.00  0.000000   0.0    100   0.000000   
    1000      0.00  0.201008   0.0    100   0.020101   
    3000     -0.54  0.845905 -54.0    100   0.084591   
0.8 25        0.00  0.000000   0.0    100   0.000000   
    100       0.00  0.000000   0.0    100   0.000000   
    1000     -0.02  0.200000  -2.0    100   0.020000   
    3000     -0.11  0.930895 -11.0    100   0.093090   
0.9 25        0.00  0.000000   0.0    100   0.000000   
    100       0.00  0.000000   0.0    100   0.000000   
    1000      0.00  0.000000   0.0    100   0.000000   
    3000      0.20  0.651339  20.0    100   0.065134   
1.0 25        0.00  0.000000   0.0    100   0.000000   
    100       0.00  0.000000   0.0    100   0.000000   
    1000      0.00  0.000000   0.0    100   0.000000   
    3000      0.00  0.000000   0.0    100   0.000000   

              Mean Episode Time in Minutes  
p   num_iter                                
0.4 25                            6.393576  
    100                          25.588154  
    1000                        237.566912  
    3000                         23.771151  
0.5 25                            6.396075  
    100                          25.585680  
    1000                        238.178359  
    3000                         38.120951  
0.6 25                            6.450283  
    100                          25.687305  
    1000                        256.371103  
    3000                         65.310226  
0.8 25                            6.464191  
    100                          25.753027  
    1000                        250.205327  
    3000                        286.300051  
0.9 25                            6.489041  
    100                          25.788930  
    1000                        241.376215  
    3000                        312.350465  
1.0 25                            6.581544  
    100                          26.087274  
    1000                        169.805646  
    3000                        310.916272

### AlphaZero Continous update with change nofication

In [12]:
exp_path = notebook_dir / "NS_FrozenLake"/ "results" / "alphazero_frozenlake_continous_update_with_change_notif_config_2024-05-29.csv"
AlphaZero_frozen_lake_continous_update_with_Notification = pd.read_csv(exp_path)
results = AlphaZero_frozen_lake_continous_update_with_Notification.groupby(["num_iter"])["reward"].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = AlphaZero_frozen_lake_continous_update_with_Notification.groupby(["num_iter"])['total_time'].agg(['mean']) / 60

results


,mean,std,sum,count,std error,Mean Episode Time in Minutes
num_iter,,,,,,
25,0.00,0.000000,0.0,100,0.000000,0.800562
100,0.00,0.000000,0.0,100,0.000000,2.925588
1000,-0.97,0.171447,-97.0,100,0.017145,9.348276
3000,-0.99,0.100000,-99.0,100,0.010000,18.358612


### PAMCTS continous update without change notification 



In [13]:
exp_path = notebook_dir /"NS_FrozenLake"/ "results" / "FrozenLake_PAMCTS_continous_update_without_change_notif_for_real_2024-05-28.csv"
FrozenLake_PAMCTS_continous_update_without_change_notif_for_real = pd.read_csv(exp_path)
# FrozenLake_PAMCTS_continous_update_without_change_notif_for_real
results = FrozenLake_PAMCTS_continous_update_without_change_notif_for_real.groupby(["num_iter","alpha"])["reward"].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = FrozenLake_PAMCTS_continous_update_without_change_notif_for_real.groupby(["num_iter","alpha"])['total_time'].agg(['mean']) / 60
results

mean       std    sum  count  std error  \
num_iter alpha                                            
25       0.25  -0.98  0.200000  -98.0    100   0.020000   
         0.50  -0.94  0.342893  -94.0    100   0.034289   
         75.00 -0.96  0.281411  -96.0    100   0.028141   
100      0.25  -1.00  0.000000 -100.0    100   0.000000   
         0.50  -0.98  0.200000  -98.0    100   0.020000   
         75.00 -1.00  0.000000 -100.0    100   0.000000   
1000     0.25  -0.98  0.200000  -98.0    100   0.020000   
         0.50  -0.90  0.438086  -90.0    100   0.043809   
         75.00 -1.00  0.000000 -100.0    100   0.000000   
3000     0.25  -0.98  0.200000  -98.0    100   0.020000   
         0.50  -0.80  0.603023  -80.0    100   0.060302   
         75.00 -1.00  0.000000 -100.0    100   0.000000   

                Mean Episode Time in Minutes  
num_iter alpha                                
25       0.25                       0.027678  
         0.50                       0.026253  
         75.00                      0.024763  
100      0.25                       0.102335  
         0.50                       0.086687  
         75.00                      0.077567  
1000     0.25                       1.192151  
         0.50                       0.684687  
         75.00                      0.280815  
3000     0.25                       2.078647  
         0.50                       1.532731  
         75.00                      0.851696

## Cartpole

In [14]:
# hyper params
exp_path = "/Users/--/Documents/--/Research/ns_gym/experiments/NS_Cartpole/results/CartpoleVanillaMCTS_withChangNotif_2024-05-21_20:55:29.csv"

cartpole_hyperparams = pd.read_csv(exp_path)

params = cartpole_hyperparams.groupby(["gamma","c","num_iter"])['reward'].agg(['mean', 'std','sum','count'])

max_reward = params["mean"].max()

params

mean         std     sum  count
gamma c        num_iter                                  
0.10  0.010000 100       464.0  132.158995  4640.0     10
      0.100000 100       352.6  136.668455  3526.0     10
      0.500000 100       500.5  146.513746  5005.0     10
      1.000000 100       359.0  150.766191  3590.0     10
      1.414214 100       392.6  133.230127  3926.0     10
      2.000000 100       602.5  556.098363  6025.0     10
0.25  0.010000 100       450.2  129.959652  4502.0     10
      0.100000 100       694.5  476.428903  6945.0     10
      0.500000 100       631.8  456.643990  6318.0     10
      1.000000 100       547.3  164.643757  5473.0     10
      1.414214 100       586.6  186.534000  5866.0     10
      2.000000 100       606.9  287.768985  6069.0     10
0.50  0.010000 100       298.5  175.051262  2985.0     10
      0.100000 100       413.4  235.684441  4134.0     10
      0.500000 100       580.0  210.532130  5800.0     10
      1.000000 100       685.1  438.213913  6851.0     10
      1.414214 100       750.2  349.191065  7502.0     10
      2.000000 100       468.8  181.794023  4688.0     10
0.75  0.010000 100         9.8    1.032796    98.0     10
      0.100000 100        12.2    2.780887   122.0     10
      0.500000 100        14.6    4.647580   146.0     10
      1.000000 100        72.8   35.798200   728.0     10
      1.414214 100       295.1  151.868693  2951.0     10
      2.000000 100       461.8  233.876891  4618.0     10
0.85  0.010000 100         9.3    0.483046    93.0     10
      0.100000 100        10.2    0.788811   102.0     10
      0.500000 100         9.6    0.699206    96.0     10
      1.000000 100        13.7    9.381424   137.0     10
      1.414214 100        12.5    2.273030   125.0     10
      2.000000 100        68.3   36.905435   683.0     10
0.99  0.010000 100         9.8    0.788811    98.0     10
      0.100000 100         9.7    0.674949    97.0     10
      0.500000 100        10.0    1.054093   100.0     10
      1.000000 100         9.9    0.875595    99.0     10
      1.414214 100         9.8    0.632456    98.0     10
      2.000000 100         9.8    1.316561    98.0     10

In [15]:

max_reward_params = params[params["mean"] == max_reward]

max_reward_params



,,,mean,std,sum,count
gamma,c,num_iter,,,,
0.5,1.414214,100,750.2,349.191065,7502.0,10


### Single transition **without** notification.

The after the first MDP decsion epoch the pole mass changes from 0.1 kg to a mass from [0.1, 1.0, 1.2, 1.3, 1.5]. The MCTS agent has access to a the MDP where pole mass == 0.1 but in the true env. the pole mass has changed. Results were generated using this file: "experiments/NS_Cartpole/NSCartpole_MCTS_single_discrete_transition_without_change_notification.py"



In [16]:
exp_path = notebook_dir / 'NS_Cartpole' / 'results' / 'CartpoleVanillaMCTS_withoutChangNotif_2024-05-21_21:50:47.csv'
cartpole_single_transition_withoutNotification = pd.read_csv(exp_path)
results = cartpole_single_transition_withoutNotification.groupby(["polemass","num_iter"])['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = cartpole_single_transition_withoutNotification.groupby(["polemass","num_iter"])['total_episode_time'].agg(['mean']) / 60


In [17]:
results

mean         std      sum  count  std error  \
polemass num_iter                                                  
0.1      50        708.08  522.140838  35404.0     50  73.841865   
         100       592.36  264.702818  29618.0     50  37.434632   
         300       660.86  352.724961  33043.0     50  49.882842   
1.0      50        667.90  393.794531  33395.0     50  55.690957   
         100       718.64  480.583649  35932.0     50  67.964791   
         300       600.90  337.124046  30045.0     50  47.676540   
1.2      50        717.70  447.498136  35885.0     50  63.285793   
         100       629.40  337.100845  31470.0     50  47.673259   
         300       595.64  374.513876  29782.0     50  52.964260   
1.3      50        552.68  271.529713  27634.0     50  38.400100   
         100       601.52  353.237692  30076.0     50  49.955353   
         300       552.84  348.475242  27642.0     50  49.281841   
1.5      50        724.84  475.593823  36242.0     50  67.259123   
         100       625.60  313.671738  31280.0     50  44.359883   
         300       641.28  356.845993  32064.0     50  50.465644   

                   Mean Episode Time in Minutes  
polemass num_iter                                
0.1      50                            1.472420  
         100                           2.442280  
         300                           8.151678  
1.0      50                            1.386213  
         100                           2.913864  
         300                           7.446955  
1.2      50                            1.420852  
         100                           2.625504  
         300                           7.026169  
1.3      50                            1.124383  
         100                           2.474727  
         300                           6.015585  
1.5      50                            1.509807  
         100                           2.402445  
         300                           5.313283

### Single transition with notification

After the first decision epoch the pole mass is updated. The MCTS agent then hass access to the most up to date MDP. 

In [18]:
exp_path = notebook_dir / "NS_Cartpole" / "results" / "CartpoleVanillaMCTS_withChangNotif_2024-05-21_21:34:20.csv"
cartpole_single_transition_with_notification = pd.read_csv(exp_path)
results = cartpole_single_transition_with_notification.groupby(["polemass","num_iter"])['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = cartpole_single_transition_with_notification.groupby(["polemass","num_iter"])['total_episode_time'].agg(['mean']) / 60
results

mean         std      sum  count  std error  \
polemass num_iter                                                  
0.1      50        745.06  430.080268  37253.0     50  60.822535   
         100       608.96  410.881480  30448.0     50  58.107416   
         300       611.78  239.512540  30589.0     50  33.872188   

                   Mean Episode Time in Minutes  
polemass num_iter                                
0.1      50                            1.531159  
         100                           2.466585  
         300                           6.460949

### MCTS Single transition with notification - changing gravity among [9.8, 20,50,500 ]

In [19]:
exp_path = notebook_dir / "NS_Cartpole" / "results" / "CartpoleVanillaMCTS_Changing_Gravity_withoutChangNotif_2024-05-16_22:43:15.csv"
cartpole_single_transition_with_notification = pd.read_csv(exp_path)
results = cartpole_single_transition_with_notification.groupby(["gravity","num_iter"])['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = cartpole_single_transition_with_notification.groupby(["gravity","num_iter"])['total_episode_time'].agg(['mean']) / 60
results

mean          std       sum  count   std error  \
gravity num_iter                                                      
9.8     25         725.38   368.907924   36269.0     50   52.171459   
        50         980.98   517.912667   49049.0     50   73.243912   
        75        1096.56   734.102900   54828.0     50  103.817828   
        100       1380.50  1107.264090   69025.0     50  156.590789   
        200       1300.50   907.372125   65025.0     50  128.321797   
        300       2248.90  1561.463537  112445.0     50  220.824291   
20.0    25         427.10   260.631826   21355.0     50   36.858906   
        50         469.06   328.299229   23453.0     50   46.428522   
        75         517.84   361.520280   25892.0     50   51.126688   
        100        573.82   396.921762   28691.0     50   56.133214   
        200       1147.90   882.175656   57395.0     50  124.758478   
        300       1262.56   841.334634   63128.0     50  118.982685   
50.0    25         157.14   105.197775    7857.0     50   14.877212   
        50         163.60    63.005669    8180.0     50    8.910347   
        75         197.28   105.313921    9864.0     50   14.893638   
        100        275.42   207.225608   13771.0     50   29.306127   
        200        398.88   282.344242   19944.0     50   39.929506   
        300        392.78   314.135372   19639.0     50   44.425450   
500.0   25           8.72     3.057543     436.0     50    0.432402   
        50           9.86     5.006159     493.0     50    0.707978   
        75          10.52     5.403098     526.0     50    0.764113   
        100         10.00     3.912304     500.0     50    0.553283   
        200         10.20     3.708787     510.0     50    0.524502   
        300         12.32     7.511732     616.0     50    1.062319   

                  Mean Episode Time in Minutes  
gravity num_iter                                
9.8     25                            0.694684  
        50                            1.939144  
        75                            3.283458  
        100                           5.542224  
        200                          10.031157  
        300                          18.947410  
20.0    25                            0.419422  
        50                            0.932704  
        75                            1.556276  
        100                           2.312814  
        200                           8.270535  
        300                          11.348108  
50.0    25                            0.156596  
        50                            0.322736  
        75                            0.586315  
        100                           1.108842  
        200                           3.225954  
        300                           4.202964  
500.0   25                            0.007501  
        50                            0.016814  
        75                            0.027504  
        100                           0.032474  
        200                           0.069775  
        300                           0.130064

### MCTS Continuos transition with notification - increasing pole mass by 0.1 each time step. 

In [20]:
exp_path = notebook_dir / "NS_Cartpole" / "results" / "CartPole_MCTS_IncreasingMass_trail_1_2024-05-27.csv"
cartpole_continuous_update_with_notification = pd.read_csv(exp_path)

results = cartpole_continuous_update_with_notification.groupby(["num_iter"])['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])

results["Mean Episode Time in Minutes"] = cartpole_continuous_update_with_notification.groupby(["num_iter"])['total_time'].agg(['mean']) / 60
results


,mean,std,sum,count,std error,Mean Episode Time in Minutes
num_iter,,,,,,
50,757.700000,466.245330,113655.0,150,38.068772,2.065447
100,693.926667,488.168046,104089.0,150,39.858754,3.893496
300,656.500000,438.856931,98475.0,150,35.832518,9.777991


### MCTS Continous transition without notification - increasing pole mass by 0.1 each time step

In [21]:
exp_path = notebook_dir / "NS_Cartpole" / "results" / "CartPole_MCTS_IncreasingMass_trial_1_without_notif_2024-05-28.csv"
cartpole_continuous_update_without_notification = pd.read_csv(exp_path)
results = cartpole_continuous_update_without_notification.groupby(["num_iter"])['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])

results["Mean Episode Time in Minutes"] = cartpole_continuous_update_without_notification.groupby(["num_iter"])['total_time'].agg(['mean']) / 60

results

,mean,std,sum,count,std error,Mean Episode Time in Minutes
num_iter,,,,,,
50,147.946667,42.515498,22192.0,150,3.471376,0.308773
100,150.020000,35.025297,22503.0,150,2.859804,0.631876
300,149.033333,36.349241,22355.0,150,2.967903,1.784670


### DDQN Continous transition without notification - increasing pole mass by 0.1 each time step

In [22]:
exp_path = notebook_dir / "NS_Cartpole" / "results" / "CartPole_DDQN_IncreasingMass_trial_1_2024-05-28.csv"
DDQN_cartpole_continuous_update_without_notification = pd.read_csv(exp_path)
results = DDQN_cartpole_continuous_update_without_notification["reward"].agg(['mean', 'std','sum','count'])

results["std error"] = results["std"] / np.sqrt(results["count"])

results["Mean Episode Time in Minutes"] = DDQN_cartpole_continuous_update_without_notification["total_time"].mean() / 60    

results.to_frame().T

,mean,std,sum,count,std error,Mean Episode Time in Minutes
reward,101.48,33.449094,15222.0,150.0,2.731107,0.017351


### PAMCTS continous transition without notification -- increasing pole mase by 0.1 each time step 

In [33]:
exp_path = notebook_dir / "NS_Cartpole" / "results" / "CartPole_PAMCTS_IncreasingMass_trial_1_without_change_notif_2024-05-28.csv"
PAMCTS_cartpole_continuous_update_without_notification = pd.read_csv(exp_path)
results = PAMCTS_cartpole_continuous_update_without_notification.groupby(["num_iter","alpha"])['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = PAMCTS_cartpole_continuous_update_without_notification.groupby(["num_iter","alpha"])['total_time'].agg(['mean']) / 60

results

mean        std      sum  count  std error  \
num_iter alpha                                                     
50       0.25   101.213333  32.548897  15182.0    150   2.657606   
         0.50   101.700000  31.482551  15255.0    150   2.570540   
         75.00   98.026667  33.285413  14704.0    150   2.717743   
100      0.25   100.320000  33.612741  15048.0    150   2.744469   
         0.50    99.800000  31.413672  14970.0    150   2.564916   
         75.00  101.933333  31.541528  15290.0    150   2.575355   
300      0.25    98.460000  33.175480  14769.0    150   2.708767   
         0.50   101.126667  33.110816  15169.0    150   2.703487   
         75.00  100.360000  31.670341  15054.0    150   2.585872   

                Mean Episode Time in Minutes  
num_iter alpha                                
50       0.25                       0.209630  
         0.50                       0.210568  
         75.00                      0.199809  
100      0.25                       0.408065  
         0.50                       0.406928  
         75.00                      0.423163  
300      0.25                       1.235850  
         0.50                       1.243384  
         75.00                      0.937210

### PAMCTS continous transition with notification -- increasing pole mase by 0.1 each time step 

In [35]:
exp_path = notebook_dir / "NS_Cartpole" / "results" / "CartPole_PAMCTS_IncreasingMass_trial_1_with_change_notif_2024-05-28.csv"
PAMCTS_cartpole_continuous_update_without_notification = pd.read_csv(exp_path)
results = PAMCTS_cartpole_continuous_update_without_notification.groupby(["num_iter","alpha"])['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = PAMCTS_cartpole_continuous_update_without_notification.groupby(["num_iter","alpha"])['total_time'].agg(['mean']) / 60

results


mean        std      sum  count  std error  \
num_iter alpha                                                     
50       0.25    99.966667  32.549274  14995.0    150   2.657637   
         0.50   104.526667  29.899765  15679.0    150   2.441306   
         75.00   96.273333  33.880345  14441.0    150   2.766319   
100      0.25   101.873333  32.155724  15281.0    150   2.625504   
         0.50    95.553333  32.399465  14333.0    150   2.645405   
         75.00   97.746667  33.255203  14662.0    150   2.715276   
300      0.25   101.066667  31.948504  15160.0    150   2.608584   
         0.50    99.593333  32.600510  14939.0    150   2.661820   
         75.00   99.533333  32.046876  14930.0    150   2.616616   

                Mean Episode Time in Minutes  
num_iter alpha                                
50       0.25                       0.196147  
         0.50                       0.206402  
         75.00                      0.184935  
100      0.25                       0.393646  
         0.50                       0.363645  
         75.00                      0.383951  
300      0.25                       1.204882  
         0.50                       1.155824  
         75.00                      0.849455

### DDQN single transition without notif. 

In [25]:
exp_path = notebook_dir / "NS_Cartpole" / "results" / "CartPole_DDQN_single_transition_increasingMass_trial_1_2024-05-30.csv"
DDQN_cartpole_single_transition_without_notification = pd.read_csv(exp_path)
results = DDQN_cartpole_single_transition_without_notification.groupby(["mass"])['reward'].agg(['mean', 'std','sum','count'])
results['std error'] = results['std'] / np.sqrt(results['count'])
results


,mean,std,sum,count,std error
mass,,,,,
0.1,136.606667,3.856306,20491.0,150,0.314866
1.0,135.533333,3.414668,20330.0,150,0.278806
1.2,135.853333,3.328516,20378.0,150,0.271772
1.3,136.026667,3.172759,20404.0,150,0.259055
1.5,135.193333,3.272207,20279.0,150,0.267175


### PAMCTS single transition without notifc

In [30]:
exp_path = notebook_dir / "NS_Cartpole" / "results" / "CartPole_PAMCTS_IncreasingMass_trial_without_change_notif_2024-05-30.csv"
PAMCTS_cartpole_single_transition_without_notification = pd.read_csv(exp_path)
print(PAMCTS_cartpole_single_transition_without_notification.columns)
results = PAMCTS_cartpole_single_transition_without_notification.groupby(["mass","alpha","num_iter"])['reward'].agg(['mean', 'std','sum','count'])
results['std error'] = results['std'] / np.sqrt(results['count'])
results

Index(['mass', 'sample_id', 'reward', 'experiment_name', 'num_iter',
       'total_time', 'seed', 'c', 'gamma', 'alpha'],
      dtype='object')


mean       std      sum  count  std error
mass alpha num_iter                                                 
0.1  0.25  50        136.146667  3.605340  20422.0    150   0.294375
           100       136.153333  3.871660  20423.0    150   0.316120
           300       136.360000  3.722379  20454.0    150   0.303931
     0.50  50        136.706667  3.627906  20506.0    150   0.296217
           100       136.220000  3.654307  20433.0    150   0.298373
           300       136.553333  3.638970  20483.0    150   0.297121
     75.00 50        136.073333  3.789949  20411.0    150   0.309448
           100       136.406667  3.578640  20461.0    150   0.292195
           300       136.520000  3.282494  20478.0    150   0.268014
1.0  0.25  50        135.953333  3.336471  20393.0    150   0.272422
           100       136.240000  3.447945  20436.0    150   0.281523
           300       135.900000  3.659593  20385.0    150   0.298804
     0.50  50        135.860000  3.561596  20379.0    150   0.290803
           100       136.166667  3.459093  20425.0    150   0.282434
           300       136.013333  3.595274  20402.0    150   0.293553
     75.00 50        136.066667  3.486632  20410.0    150   0.284682
           100       136.426667  3.374493  20464.0    150   0.275526
           300       135.246667  3.465916  20287.0    150   0.282991
1.2  0.25  50        136.306667  3.223127  20446.0    150   0.263167
           100       135.960000  3.337563  20394.0    150   0.272511
           300       135.913333  3.369697  20387.0    150   0.275135
     0.50  50        135.473333  3.247501  20321.0    150   0.265157
           100       135.726667  3.540554  20359.0    150   0.289085
           300       135.166667  3.622111  20275.0    150   0.295744
     75.00 50        135.353333  3.061203  20303.0    150   0.249946
           100       135.866667  3.375129  20380.0    150   0.275578
           300       135.546667  3.286308  20332.0    150   0.268326
1.3  0.25  50        135.693333  3.383629  20354.0    150   0.276272
           100       135.920000  3.554976  20388.0    150   0.290263
           300       135.773333  3.431294  20366.0    150   0.280164
     0.50  50        136.060000  3.616201  20409.0    150   0.295262
           100       135.686667  3.446887  20353.0    150   0.281437
           300       135.540000  3.549516  20331.0    150   0.289817
     75.00 50        134.946667  3.570548  20242.0    150   0.291534
           100       135.600000  3.636135  20340.0    150   0.296889
           300       135.773333  3.334076  20366.0    150   0.272226
1.5  0.25  50        135.933333  3.249505  20390.0    150   0.265321
           100       135.573333  3.742279  20336.0    150   0.305556
           300       135.080000  3.265685  20262.0    150   0.266642
     0.50  50        135.686667  3.552363  20353.0    150   0.290049
           100       135.640000  3.225902  20346.0    150   0.263394
           300       135.260000  3.308379  20289.0    150   0.270128
     75.00 50        135.513333  3.255840  20327.0    150   0.265838
           100       135.873333  3.065935  20381.0    150   0.250333
           300       135.913333  3.325591  20387.0    150   0.271533

# Mountain Car

## Hyper param search



In [27]:
exp_path = notebook_dir / "NS_Mountain_Car"/ "results" / "MCTS_MountainCar_hyperparameter_search_2024-05-23.csv"

mountain_car_hyperparams = pd.read_csv(exp_path)
mountain_car_hyperparams


,sample_id,reward,experiment_name,num_iter,total_time,seed,c,gamma
0,70,-200.0,MCTS_MountainCar_hyperparameter_search_2024-05-23,100,126.618293,42509,0.8,0.80
1,37,-200.0,MCTS_MountainCar_hyperparameter_search_2024-05-23,100,126.916794,48713,0.8,0.50
2,47,-200.0,MCTS_MountainCar_hyperparameter_search_2024-05-23,100,126.931805,56092,0.5,0.99
3,9,-200.0,MCTS_MountainCar_hyperparameter_search_2024-05-23,100,127.028807,58505,0.5,0.99
4,32,-200.0,MCTS_MountainCar_hyperparameter_search_2024-05-23,100,127.081158,32573,0.8,0.80
...,...,...,...,...,...,...,...,...
2395,22,-200.0,MCTS_MountainCar_hyperparameter_search_2024-05-23,100,107.468034,68046,5.0,0.99
2396,93,-200.0,MCTS_MountainCar_hyperparameter_search_2024-05-23,100,112.441218,63411,5.0,1.00
2397,55,-200.0,MCTS_MountainCar_hyperparameter_search_2024-05-23,100,109.112278,12933,5.0,1.00
2398,41,-200.0,MCTS_MountainCar_hyperparameter_search_2024-05-23,100,110.174428,79533,5.0,0.99


In [28]:
results = mountain_car_hyperparams.groupby(["gamma","c","num_iter"])['reward'].agg(['mean', 'std','sum','count'])
results["std error"] = results["std"] / np.sqrt(results["count"])
results["Mean Episode Time in Minutes"] = mountain_car_hyperparams.groupby(["gamma","c","num_iter"])['total_time'].agg(['mean']) / 60
results

mean  std      sum  count  std error  \
gamma c        num_iter                                          
0.50  0.500000 100      -200.0  0.0 -20000.0    100        0.0   
      0.800000 100      -200.0  0.0 -20000.0    100        0.0   
      1.000000 100      -200.0  0.0 -20000.0    100        0.0   
      1.414214 100      -200.0  0.0 -20000.0    100        0.0   
      2.000000 100      -200.0  0.0 -20000.0    100        0.0   
      5.000000 100      -200.0  0.0 -20000.0    100        0.0   
0.80  0.500000 100      -200.0  0.0 -20000.0    100        0.0   
      0.800000 100      -200.0  0.0 -20000.0    100        0.0   
      1.000000 100      -200.0  0.0 -20000.0    100        0.0   
      1.414214 100      -200.0  0.0 -20000.0    100        0.0   
      2.000000 100      -200.0  0.0 -20000.0    100        0.0   
      5.000000 100      -200.0  0.0 -20000.0    100        0.0   
0.99  0.500000 100      -200.0  0.0 -20000.0    100        0.0   
      0.800000 100      -200.0  0.0 -20000.0    100        0.0   
      1.000000 100      -200.0  0.0 -20000.0    100        0.0   
      1.414214 100      -200.0  0.0 -20000.0    100        0.0   
      2.000000 100      -200.0  0.0 -20000.0    100        0.0   
      5.000000 100      -200.0  0.0 -20000.0    100        0.0   
1.00  0.500000 100      -200.0  0.0 -20000.0    100        0.0   
      0.800000 100      -200.0  0.0 -20000.0    100        0.0   
      1.000000 100      -200.0  0.0 -20000.0    100        0.0   
      1.414214 100      -200.0  0.0 -20000.0    100        0.0   
      2.000000 100      -200.0  0.0 -20000.0    100        0.0   
      5.000000 100      -200.0  0.0 -20000.0    100        0.0   

                         Mean Episode Time in Minutes  
gamma c        num_iter                                
0.50  0.500000 100                           2.269702  
      0.800000 100                           2.254777  
      1.000000 100                           2.293076  
      1.414214 100                           2.293202  
      2.000000 100                           2.293294  
      5.000000 100                           2.224888  
0.80  0.500000 100                           2.271982  
      0.800000 100                           2.252528  
      1.000000 100                           2.300976  
      1.414214 100                           2.292331  
      2.000000 100                           2.294213  
      5.000000 100                           2.223161  
0.99  0.500000 100                           2.255736  
      0.800000 100                           2.288334  
      1.000000 100                           2.289831  
      1.414214 100                           2.292187  
      2.000000 100                           2.243227  
      5.000000 100                           2.219049  
1.00  0.500000 100                           2.257963  
      0.800000 100                           2.297389  
      1.000000 100                           2.299051  
      1.414214 100                           2.298783  
      2.000000 100                           2.238602  
      5.000000 100                           2.231270